In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Read Dataframe to get data
df = pd.read_csv('AAPL_2023.csv')
price_column = df['close']
unchanged_stock_prices = price_column.to_numpy() 
stock_prices = (unchanged_stock_prices - np.min(unchanged_stock_prices)) / (np.max(unchanged_stock_prices) - np.min(unchanged_stock_prices))

# Convert to PyTorch tensors and reshape for LSTM
sequence_length = 8
x_train, y_train = [], []

for i in range(len(stock_prices) - sequence_length):
    x_train.append(stock_prices[i:i + sequence_length])
    y_train.append(stock_prices[i + sequence_length])

# Reshape for LSTM
x_train = torch.tensor(x_train, dtype=torch.float32).view(-1, sequence_length, 1)  
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

C:\Users\Colin.Mcbride.SERN\AppData\Local\Temp\ipykernel_19852\3051339647.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  x_train = torch.tensor(x_train, dtype=torch.float32).view(-1, sequence_length, 1)


In [2]:
# Define LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Create hidden layer and cell layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) 
        
        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))
        
        # Fully connected layer
        out = self.fc(out[:, -1, :])
        return out

In [3]:
# Initialize model, loss, and optimizer
input_size = 1        # Each week has 1 stock price
hidden_size = 64      # Number of LSTM units
output_size = 1       # Predicting 1 stock price
num_layers = 2        # Double LSTM layer
learning_rate = 0.001
num_epochs = 100

model = LSTMModel(input_size, hidden_size, output_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Training
for epoch in range(num_epochs):
    model.train()
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print statement for debug and loss
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [5]:
# Predicting entire 2024
model.eval()
last_sequence = x_train[-1].unsqueeze(0)
future_predictions = []
for _ in range(365):
    with torch.no_grad():
        next_price = model(last_sequence)  # Predict next price
    
    future_predictions.append(next_price.item())
    next_sequence = torch.cat((last_sequence[:, 1:, :], next_price.view(1, 1, 1)), dim=1)
    last_sequence = next_sequence

# Unnormalize values
future_predictions = np.array(future_predictions)
future_predictions = np.round(
    future_predictions * (np.max(unchanged_stock_prices) - np.min(unchanged_stock_prices)) + np.min(unchanged_stock_prices), 2
)

# Output predictions
print("Predicted stock prices for the next year:", future_predictions)